### I) define hyper_parameters, datasets, and IO operations

In [ ]:
# convention : all new inputs parameters for the notebbok through papermil,
# that are progressivelt added have a default value, equal to the one tha would be used in
# order obtain the same results of scripts

# two types of inputs: 
# inputs that influences the training (e.g hyper-paramteres, dataset set and splitting)
# inputs that controls state of training (reset it or load from)
import torch
from pathlib import Path


In [ ]:
path_model_and_dependencies = paths.path_model_and_dependencies

if train_state_control.load_from_backup:
    assert Path(path_model_and_dependencies).exists(),f"back_up at path : {path_model_and_dependencies} doesn't exists"
    back_up = torch.load(path_model_and_dependencies)
    back_up_train_state = back_up["model_training_state"]
    # add here routines to check the if the given parameters are coherent ith the values from the back
    # up maybe using the notebookRunner class, that would handle this resposability

init or load the model

In [ ]:
from translation_machine.models import transformer_mod
from translation_machine import sentence_mod

from pathlib import Path
import torch

if train_state_control.load_from_backup:
    model_inputs = back_up_train_state["model_inputs"]
else:
    model_inputs = {
        "d_model":simple_hp.d_model,
        "vocab_src":sentence_mod.EnglishSentence.vocab,
        "vocab_tgt":sentence_mod.FrenchSentence.vocab,
    }

model = transformer_mod.TransformerForSeq2Seq(**model_inputs)
    
if train_state_control.load_from_backup:
    # once we choose to load the model, it is always loaded from backup
    model.load_state_dict(back_up_train_state["model_params"])

In [ ]:
#optimizer.link(model.parameters())

In [ ]:
optimizer = opt_params.unlinked_optimizer(model.parameters())
scheduler = opt_params.unlinked_scheduler(optimizer)

if train_state_control.load_from_backup and train_state_control.restore_optimizer:
    optimizer.load_state_dict(back_up["model_training_state"]["optimizer"])
    scheduler.load_state_dict(back_up["model_training_state"]["scheduler"])

In [ ]:
if train_state_control.load_from_backup:
    losses = back_up["results"]["losses"]
    metrics = back_up["results"]["metrics"]
else:
    losses = {"train":[],"val":[]}
    metrics = {"train":[],"val":[]}


In [ ]:
from translation_machine.models import transformer_mod
from translation_machine import sentence_mod

if train_state_control.load_from_backup:
    model_inputs = back_up["model_training_state"]["model_inputs"]
else:
    model_inputs = {
        "d_model":simple_hp.d_model,
        "vocab_src":sentence_mod.EnglishSentence.vocab,
        "vocab_tgt":sentence_mod.FrenchSentence.vocab,
    }

model = transformer_mod.TransformerForSeq2Seq(**model_inputs)

if train_state_control.load_from_backup:
    model.load_state_dict(back_up_train_state["model_params"])
    print("model loaded")

# II) load the vocabulary

In [ ]:
from translation_machine import dataset_mod,sentence_mod

import torch,numpy as np

language_info = torch.load(paths.path_language_info)

vocab_french = language_info["french"]["vocab"]
vocab_english = language_info["english"]["vocab"]


len(vocab_french),len(vocab_english)

# III) Load the dataset

In [ ]:
if dset_truncation.limit_length is None:
    dset_truncation.limit_length = language_info["limit_length"]
else:
    dset_truncation.limit_length = min(language_info["limit_length"],dset_truncation.limit_length)
    
whole_dataset = dataset_mod.DatasetFromTxt(paths.path_dataset)

idxs_whole = np.arange(dset_truncation.limit_length)
dataset = torch.utils.data.Subset(whole_dataset,idxs_whole)

dataset = list(dataset_mod.SentenceDataSet(dataset,sentence_type_src=sentence_mod.EnglishSentence,sentence_type_dst=sentence_mod.FrenchSentence))
len(dataset)

### IV) preparation of notebook params for serialization (for the purpose of associating the run to results of training), 

In [ ]:
max_length_from_file = False
if max_length_from_file:
    max_length_french = language_info["french"]["max_sentence_train_val"]
    max_length_english = language_info["english"]["max_sentence_train_val"]
else:# get max length from current dataset, which is prefered
    import itertools
    tmp = [(len(el[0]),len(el[1])) for el in dataset]
    a,b = zip(*tmp)
    max_length_english  = max(a)
    max_length_french = max(b)
    
max_length_english,max_length_french

### V) dataset splitting

In [ ]:
# Remark : the responsability to split the dataset is done outside of this notebook
from pathlib import Path

if dset_truncation.use_splitting:
    path_dataset_splitting = paths.path_dataset_splitting
    path_idxs_train = str(Path(path_dataset_splitting).joinpath("idxs_train.npy"))
    path_idxs_val = str(Path(path_dataset_splitting).joinpath("idxs_val.npy"))
    path_idxs_test = str(Path(path_dataset_splitting).joinpath("idxs_test.npy"))

    idxs_train = np.load(path_idxs_train)
    idxs_val = np.load(path_idxs_val)
    idxs_test = np.load(path_idxs_test)

    idxs_train,idxs_val,idxs_test = [[idx for idx in idxs if idx<len(whole_dataset)] for idxs in [idxs_train,idxs_val,idxs_test]]
    idxs_train = list(set(idxs_whole).intersection(set(idxs_train)))
    idxs_val = list(set(idxs_whole).intersection(set(idxs_val)))
    idxs_test = list(set(idxs_whole).intersection(set(idxs_test)))
    
else:
    idxs_train = idxs_whole
    idxs_val = idxs_whole
    idxs_test = idxs_whole

train_dataset = torch.utils.data.Subset(dataset,idxs_train)
val_dataset = torch.utils.data.Subset(dataset,idxs_val)
test_dataset = torch.utils.data.Subset(dataset,idxs_test)
len(train_dataset),len(val_dataset),len(test_dataset)

### VI) dataloader construction

In [ ]:
from translation_machine import collate_fn_mod
from torch.utils.data import DataLoader

import torch
import numpy as np

collate_fn = collate_fn_mod.get_collate_fn(max_length_english,max_length_french)

train_data_loader = DataLoader(train_dataset,batch_size=simple_hp.batch_size,
                               shuffle=True,collate_fn=collate_fn)
val_data_loader = DataLoader(val_dataset,batch_size=simple_hp.batch_size,
                             shuffle=True,collate_fn=collate_fn)

In [ ]:
from torch import optim,nn


baseline_loss = nn.CrossEntropyLoss(reduction="sum")
